In [1]:
import httpx as rq
import pandas as pd


# database

In [2]:
from sqlalchemy import create_engine, Table, Column, MetaData, Integer, String


engine = create_engine("sqlite:///news.db", echo=True)

meta = MetaData()

news = Table(
    "news",
    meta,
    Column("author", String),
    Column("title", String),
    Column("description", String),
    Column("url", String, primary_key=True),
    Column("urlToImage", String),
    Column("publishedAt", String),
    Column("content", String),
    Column("source_id", String),
    Column("source_name", String),
)

meta.create_all(engine)


2024-04-02 14:06:42,115 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-02 14:06:42,116 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("news")
2024-04-02 14:06:42,116 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-02 14:06:42,118 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
existing_data = pd.read_sql("SELECT * FROM news", engine)

# Fetch new data
params = {
    "q": "Apple",
    "sortBy": "popularity",
    "apiKey": "45a541a896864ea0becf962b10f5a79e",
}
response = rq.get("https://newsapi.org/v2/top-headlines", params=params)
new_data = pd.json_normalize(response.json()["articles"], sep="_")

# Filter out data that already exists
new_data = new_data[~new_data["url"].isin(existing_data["url"])]

# Append new data to existing data
merged_data = pd.concat([existing_data, new_data])

# Update database
merged_data.to_sql("news", engine, if_exists="replace")

# Send webhook with new information
# Implement webhook sending logic here
print("Webhook sent with new information:", new_data.shape[0], "new articles.")

2024-04-02 14:06:49,493 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-02 14:06:49,496 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM news")
2024-04-02 14:06:49,497 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-02 14:06:49,497 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM news")
2024-04-02 14:06:49,497 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-02 14:06:49,503 INFO sqlalchemy.engine.Engine SELECT * FROM news
2024-04-02 14:06:49,504 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-02 14:06:49,520 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-02 14:06:51,592 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-02 14:06:51,601 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("news")
2024-04-02 14:06:51,602 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-02 14:06:51,603 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("news")
2024-04-02 14:06:51,603 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-02 14:06:51